## Create Datasets

In [4]:
import sys
sys.path.append('../')
from src.data.preprocess import load_text_data, load_image_data, attach_image_data, save_preprocessed_data

In [ ]:
# Paths to your text and image data
text_data_paths = ['../data/raw/train_sent_emo.csv', '../data/raw/dev_sent_emo.csv', '../data/raw/test_sent_emo.csv']
image_data_paths = ['../data/raw/train.pkl', '../data/raw/dev.pkl', '../data/raw/test.pkl']

In [ ]:
# Load and preprocess text data
train_df, dev_df, test_df = load_and_preprocess_data(text_data_paths)

# Load image data
image_data = [load_image_data(path) for path in image_data_paths]

# Attach image data to your DataFrames
text_data_frames = [train_df, dev_df, test_df]
text_data_frames = attach_image_data(text_data_frames, image_data)
train_df, dev_df, test_df = text_data_frames

# Save the preprocessed data (optional)
save_paths = ['../data/processed/train_full.pkl', '../data/processed/dev_full.pkl', '../data/processed/test_full.pkl']
save_preprocessed_data(text_data_frames, save_paths)

## Models

In [7]:
from src.models.train_model import train_text_only_model, train_image_only_model, train_multi_modal_model
from src.models.evaluate_model import evaluate_model

In [ ]:
# Transform target into dummy vector
y_train = pd.get_dummies(train_df['Emotion']).to_numpy()
y_dev = pd.get_dummies(dev_df['Emotion']).to_numpy()
y_test = pd.get_dummies(test_df['Emotion']).to_numpy()

In [ ]:
# Define BERT pre-preprocessing
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

# Define BERT encoder
bert_layers = 12
bert_units = 768
bert_heads = 12
bert_encoder = f'https://tfhub.dev/tensorflow/bert_en_uncased_L-{bert_layers}_H-{bert_units}_A-{bert_heads}/4'

### Text-only Model

In [ ]:
# Identify text features
X_train = train_df['input']
X_test = test_df['input']

In [ ]:
# Train model
text_model = train_text_only_model(X_train, y_train, bert_preprocess, bert_encoder)

In [ ]:
# Evaluate model
accuracy, f1, conf_matrix = evaluate_model(text_model, X_test, y_test)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

### Image-only Model

In [ ]:
# Identify image features
X_train = np.array([np.concatenate([np.array(row['first_frame']), np.array(row['middle_frame']), np.array(row['last_frame'])]) for index, row in train_df.iterrows()])
X_test = np.array([np.concatenate([np.array(row['first_frame']), np.array(row['middle_frame']), np.array(row['last_frame'])]) for index, row in test_df.iterrows()])

# Scale image features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Train model
image_model = train_image_only_model(X_train, y_train)

In [ ]:
# Evaluate model
accuracy, f1, conf_matrix = evaluate_model(image_model, X_test, y_test)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

### Multi-modal model

In [ ]:
# Identify text features
X_train_text = train_df['input']
X_test_text = test_df['input']

# Identify image features
X_train_image = np.array([np.concatenate([np.array(row['first_frame']), np.array(row['middle_frame']), np.array(row['last_frame'])]) for index, row in train_df.iterrows()])
X_test_image = np.array([np.concatenate([np.array(row['first_frame']), np.array(row['middle_frame']), np.array(row['last_frame'])]) for index, row in test_df.iterrows()])

# Scale image features
scaler = StandardScaler()
X_train_image = scaler.fit_transform(X_train_image)
X_test_image = scaler.transform(X_test_image)

# Concatenate features
X_train = [X_train_text, X_train_image]
X_test = [X_test_text, X_test_image]

In [ ]:
# Train model
multi_modal_model = train_multi_modal_model(X_train, y_train, bert_preprocess, bert_encoder)

In [ ]:
# Evaluate model
accuracy, f1, conf_matrix = evaluate_model(multi_modal_model, X_test, y_test)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)